In [35]:
import re

def section_aware_split(text: str, max_chunk_len: int = 1500) -> list:
    """
    Split text into semantic chunks using section headers and preserve structure for Q&A retrieval.
    Adds [SECTION] tags for better metadata extraction and table grouping.
    """
    # Match numbered headers (e.g., 1., 1.1) or Thai/English titles
    section_pattern = re.compile(
        r"(?:^|\n)([0-9]+\.[0-9]*[^\n]{0,80}|^ธุรกิจ[^\n]+|^ศูนย์[^\n]+|^ค่าสาธารณูปโภค[^\n]*)", re.MULTILINE)

    parts = section_pattern.split(text)
    grouped = []

    for i in range(1, len(parts), 2):
        header = parts[i].strip()
        body = parts[i + 1].strip() if i + 1 < len(parts) else ""
        grouped.append(f"[SECTION] {header}\n{body}")

    # Combine multiple grouped sections into chunks within max_chunk_len
    final_chunks = []
    current_chunk = ""
    for section in grouped:
        if len(current_chunk) + len(section) > max_chunk_len:
            if current_chunk:
                final_chunks.append(current_chunk.strip())
            current_chunk = section
        else:
            current_chunk += "\n\n" + section

    if current_chunk:
        final_chunks.append(current_chunk.strip())

    return final_chunks

In [36]:
import os
import nest_asyncio

from llama_index.core import Document, VectorStoreIndex, StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core.vector_stores import VectorStoreQueryResult
from qdrant_client import QdrantClient, AsyncQdrantClient
from llama_index.embeddings.cohere import CohereEmbedding
from llama_index.core import Settings
from typing import List
from dotenv import load_dotenv
import json
import os
nest_asyncio.apply()
load_dotenv(dotenv_path=".env.dev")

True

In [37]:
from docx import Document

def extract_tables_as_markdown(docx_path):
    doc = Document(docx_path)
    markdown_tables = []
    for table in doc.tables:
        rows = []
        for row in table.rows:
            cells = [cell.text.strip() for cell in row.cells]
            rows.append("| " + " | ".join(cells) + " |")
        if rows:
            header = rows[0]
            separator = "| " + " | ".join(["---"] * len(table.columns)) + " |"
            markdown_table = "\n".join([header, separator] + rows[1:])
            markdown_tables.append(markdown_table)
    return markdown_tables

In [38]:
from llama_index.readers.file import DocxReader
from llama_index.core.schema import Document
import os
import glob
import re

# ——————————————
# CONFIGURATION
# ——————————————
DOCX_FOLDER = "documents/"
SHAREPOINT_BASE_URL = "https://cpaxtra.sharepoint.com/sites/forms-library"

# ——————————————
# SECTION TITLE HELPER
# ——————————————
def extract_section_title(chunk: str) -> str:
    """
    Extract section title from chunk marked as [SECTION] ... or fallback to first line.
    """
    match = re.search(r"\[SECTION\] (.*?)\n", chunk)
    if match:
        return match.group(1).strip()
    # fallback to first non-empty line
    lines = [line.strip() for line in chunk.splitlines() if line.strip()]
    return lines[0] if lines else "unknown"

# ——————————————
# STEP 1: Discover all .docx files
# ——————————————
all_paths = glob.glob(os.path.join(DOCX_FOLDER, "*.docx"))
print(f"Found {len(all_paths)} .docx file(s):")
for p in all_paths:
    print("  •", p)

# ——————————————
# STEP 2: Load each DOCX and wrap as Document
# ——————————————
reader = DocxReader()
raw_documents = []
for file_path in all_paths:
    docx_pages = reader.load_data(file_path)
    for page_obj in docx_pages:
        raw_documents.append(
            Document(
                text=page_obj.text,
                metadata={"source": os.path.basename(file_path)}
            )
        )

print(f"Loaded {len(raw_documents)} raw Document(s) from all .docx files.")

# ——————————————
# STEP 3: Chunk each Document semantically with metadata
# ——————————————
nodes = []
for doc in raw_documents:
    file_name = doc.metadata.get("source", "")
    attachment_link = f"{SHAREPOINT_BASE_URL}/{file_name}"
    section_chunks = section_aware_split(doc.text)

    for i, chunk in enumerate(section_chunks):
        section_title = extract_section_title(chunk)
        nodes.append(
            Document(
                text=chunk,
                metadata={
                    **doc.metadata,
                    "chunk_id": i,
                    "section_title": section_title,
                    "attachment_link": attachment_link,
                }
            )
        )

print(f"After splitting, we have {len(nodes)} chunked Documents (nodes).")

# ——————————————
# FINAL: Assign to `documents` so rest of pipeline stays unchanged
# ——————————————
documents = nodes

Found 8 .docx file(s):
  • documents/FA-G-15_ Trade Supplier Registration and Payment Policy.docx
  • documents/อำนาจอนุมัติรายจ่ายสำหรับ Purchase Requisition_แปลงตาราง.docx
  • documents/FA-G-02 (T) Staff Expense Reimbursement _15012025 (Chat bot).docx
  • documents/CPAX-FN-005_Project Investment_TH_Final_Narrative_Chatbot.docx
  • documents/ระเบียบปฏิบัติ เรื่อง อำนาจอนุมัติ Level of Authorization_แปลงตาราง (1).docx
  • documents/FA-G-17  Tenant Selection and Debt Collection_Chatbot) (1).docx
  • documents/FA-G-07 Non-Trade Supplier (Chat bot) (1).docx
  • documents/FA-B2B-01 Credit Management for B2B_10032025_for sign_chatbot_Final_v1.docx
Loaded 8 raw Document(s) from all .docx files.
After splitting, we have 26 chunked Documents (nodes).


In [39]:
from llama_index.core import StorageContext

## Setup Cohear Embedding service

In [40]:
# … (no need to call load_dotenv() here) …

# Hard-code your key and model ID:
COHEAR_KEY      = "1g4IppS8Bq9OocS1c57AXaTIrzfNnGlUdq0mt4LF"
COHEAR_MODEL_ID = "embed-multilingual-light-v3.0"

print("🔑 Using Cohere key:   ", COHEAR_KEY)
print("🔢 Using Cohere model: ", COHEAR_MODEL_ID)

embed_model = CohereEmbedding(
    api_key=COHEAR_KEY,
    model_name=COHEAR_MODEL_ID,
    input_type="search_document",
    embedding_type="float",
)

Settings.chunk_size = 1024

🔑 Using Cohere key:    1g4IppS8Bq9OocS1c57AXaTIrzfNnGlUdq0mt4LF
🔢 Using Cohere model:  embed-multilingual-light-v3.0


## Innitiates VectorStore database (Qdrant)

In [41]:
from qdrant_client import QdrantClient
from llama_index.vector_stores.qdrant import QdrantVectorStore
import os

# Initialize Qdrant client with HTTP (not gRPC)
client = QdrantClient(
    url="http://localhost:6433",  # Using HTTP endpoint exposed by Docker
    api_key=os.getenv("QDRANT_API_KEY"),
    prefer_grpc=False,            # Disable gRPC to avoid connection issues
    timeout=60,
    check_compatibility=False     # Suppress version mismatch warning
)

# Load collection name from environment
collection_name = os.getenv("QDRANT_COLLECTION_NAME")

# Delete collection if it exists
if client.collection_exists(collection_name):
    client.delete_collection(collection_name)

# Create Qdrant vector store with hybrid search enabled
vector_store = QdrantVectorStore(
    collection_name=collection_name,
    client=client,
    enable_hybrid=True,
    batch_size=20,
    prefer_grpc=False             # Match client setting
)

/var/folders/z5/qxv1ygl57v34c8nqkcj1m3200000gn/T/ipykernel_24187/3254164064.py:6: UserWarning: Api key is used with an insecure connection.
  client = QdrantClient(


## Start embedding process.... into vector database

In [42]:
# ✅ Hardcoded API key and model config (no .env loading)
COHERE_API_KEY = "4mXlJe9QpUGIXxftxHgyqjw81TODwvQ2NsWq57ut"
COHERE_MODEL_ID = "embed-multilingual-light-v3.0"  # <-- replace with your actual model if different

QDRANT_URL = "http://localhost:6334"
QDRANT_API_KEY = None  # Set this to your Qdrant key if needed
COLLECTION_NAME = "my_collection"

print("✅ COHERE_API_KEY loaded.")

# ✅ Initialize Cohere embed model
embed_model = CohereEmbedding(
    cohere_api_key=COHERE_API_KEY,
    model_name=COHERE_MODEL_ID,
    input_type="search_document",
    embedding_type="float",
)

# ✅ Build index from documents
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents=documents,
    embed_model=embed_model,
    storage_context=storage_context,
)

✅ COHERE_API_KEY loaded.


## Try to retrive relavent nodes with question.

In [43]:
embed_model = CohereEmbedding(
    api_key=os.getenv("COHERE_API_KEY"),
    model_name=os.getenv("COHERE_MODEL_ID"),
    input_type="search_query",
    embedding_type="float",
)

search_query_retriever = index.as_retriever()

search_query_retrieved_nodes = search_query_retriever.retrieve(
"Do all Walmart locations offer scan & go?"
)

In [44]:
from llama_index.core.response.notebook_utils import display_source_node
for n in search_query_retrieved_nodes:
    display_source_node(n, source_length=2000)

**Node ID:** 62868c9f-6e21-4f0d-9d57-7ee56eb9b0d7<br>**Similarity:** 0.05706907<br>**Text:** [SECTION] 4.2	ตรวจสอบความครบถ้วนของเอกสารประกอบ หากไม่ครบถ้วนจะระบุเอกสารที่ขาดหายไป


[SECTION] 4.3	ตรวจสอบสถานะทางการเงิน และทำ Due diligence ตามหลักเกณฑ์ (Criteria) ที่ระบุใน เอ
กสารแนบ ข<br>

**Node ID:** f1924cb5-c00b-47aa-9cad-bb185a371a14<br>**Similarity:** 0.052682422<br>**Text:** 000,000 บาท จะต้องได้รับอนุมัติจาก Group CEO

	

	ธุรกิจค้าปลีก ส่วนงาน Retail Operation ค่าใช้จ่ายทั่วไป

	- มูลค่ารายการไม่เกิน 10,000 บาท จะต้องได้รับอนุมัติจากตำแหน่ง Store Manager ขึ้นไป

	- มูลค่ารายการ 10,001 - 30,000 บาท จะต้องได้รับอนุมัติจากตำแหน่ง Area General Manager ขึ้นไป

	- มูลค่ารายการ 30,001 - 100,000 บาท จะต้องได้รับอนุมัติจากตำแหน่ง Director - Region Operations (RD) ขึ้นไป

	- มูลค่ารายการ 100,001 - 3,000,000 บาท จะต้องได้รับอนุมัติจาก Director ขึ้นไป

	- มูลค่ารายการ 3,000,001 - 20,000,000 บาท จะต้องได้รับอนุมัติจาก Senior Director ขึ้นไป

	- มูลค่ารายการ 20,000,001 - 25,000,000 บาท จะต้องได้รับอนุมัติจาก Chief (Division) ขึ้นไป

	- มูลค่ารายการ 25,000,001 - 28,000,000 บาท จะต้องได้รับอนุมัติจาก Chief (Function) ขึ้นไป

	- มูลค่ารายการ 28,000,001 - 30,000,000 บาท จะต้องได้รับอนุมัติจาก Group Chief (Function) Officer ขึ้นไป

	- มูลค่ารายการ 30,000,001 - 100,000,000 บาท จะต้องได้รับอนุมัติจาก CEO

	- มูลค่ารายการมากกว่า 100,000,000 บาท จะต้องได้รับอนุมัติจาก Group CEO

	

	ธุรกิจค้าปลีก ส่วนงาน Retail Operation ค่าสาธารญูปโภค Hypermarket

	- มูลค่ารายการไม่เกิน 100,000 บาท จะต้องได้รับอนุมัติจากตำแหน่ง Store Manager ขึ้นไป

	- มูลค่ารายการ 100,001 - 200,000 บาท จะต้องได้รับอนุมัติจากตำแหน่ง Area General Manager ขึ้นไป

	- มูลค่ารายการ 200,001 - 300,<br>